# Dependancies:

In [1]:
import os
from sys import exit, exc_info, argv
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install git+https://github.com/slremy/netsapi --user --upgrade

from netsapi.challenge import *
# For a given environment, evaluate a policy by applying its evaluateReward method


  Cloning https://github.com/slremy/netsapi to /private/var/folders/nh/8jkcnd190cd60q13l2ds0gh40000gn/T/pip-req-build-zn1chfeh
  Running command git clone -q https://github.com/slremy/netsapi /private/var/folders/nh/8jkcnd190cd60q13l2ds0gh40000gn/T/pip-req-build-zn1chfeh
  Stored in directory: /private/var/folders/nh/8jkcnd190cd60q13l2ds0gh40000gn/T/pip-ephem-wheel-cache-r_u9p1ds/wheels/9e/73/c9/86a9cc2460e11b3ce5b0a5ebd2d9d332a68afe0941659967fa
Successfully built netsapi
  Found existing installation: netsapi 1.1
    Uninstalling netsapi-1.1:
      Successfully uninstalled netsapi-1.1


In [16]:
class RemyGA:
    '''
    Simple Genetic Algorithm. 
    https://github.com/slremy/estool/
    '''
    def __init__(self, num_params,      # number of model parameters
               random_individuals_fcn,
               mutate_fcn,
               immigrant_ratio=.2,     # percentage of new individuals
               sigma_init=0.1,        # initial standard deviation
               sigma_decay=0.999,     # anneal standard deviation
               sigma_limit=0.01,      # stop annealing if less than this
               popsize=256,           # population size
               elite_ratio=0.1,       # percentage of the elites
               forget_best=False,     # forget the historical best elites
               weight_decay=0.01,     # weight decay coefficient
              ):

        self.num_params = num_params
        self.sigma_init = sigma_init
        self.sigma_decay = sigma_decay
        self.sigma_limit = sigma_limit
        self.popsize = popsize
        self.random_individuals_fcn = random_individuals_fcn
        self.mutate_fcn = mutate_fcn
        self.solutions = None

        self.elite_ratio = elite_ratio
        self.elite_popsize = int(self.popsize * self.elite_ratio)
        self.immigrant_ratio = immigrant_ratio
        self.immigrant_popsize = int(self.popsize * self.immigrant_ratio)

        self.sigma = self.sigma_init
        self.elite_params = np.zeros((self.elite_popsize, self.num_params))
        #self.elite_rewards = np.zeros(self.elite_popsize)
        self.best_param = np.zeros(self.num_params)
        self.best_reward = 0
        self.reward_pdf = np.zeros(self.popsize+1)
        self.solutions = np.zeros((self.popsize, self.num_params))
        self.first_iteration = True
        self.forget_best = forget_best
        self.weight_decay = weight_decay

    def rms_stdev(self):
        return self.sigma # same sigma for all parameters.

    def ask(self, process=lambda x:x):
        '''returns a list of parameters'''
        self.epsilon = np.random.randn(self.popsize, self.num_params) * self.sigma


        def mate(a, b):
            #single point crossover
            c = np.copy(a)
            idx = np.where(np.random.rand((c.size)) > 0.5)
            c[idx] = b[idx]
            return c

        def crossover(a,b):
            cross_point = int((self.num_params-1)*np.random.rand(1));
            c = np.append(a[:cross_point], b[cross_point:self.num_params]);
            return c

        index_array = np.arange(self.popsize)
        if self.first_iteration:
            self.solutions = process(self.random_individuals_fcn(self.popsize,self.num_params))
        else:
            #intialize the index list for "mating" chromosomes
            childrenIDX = range(self.popsize - self.elite_popsize - self.immigrant_popsize);
            selected = np.arange(2*len(childrenIDX));
            for i in range(len(selected)):
                testNo = 1;
                #Choose a parent
                while self.reward_pdf[testNo] < np.random.rand():
                    testNo = testNo + 1;
                selected[i] = index_array[testNo];
            children = []
            for i in range(len(childrenIDX)):
                chromosomeA = self.solutions[selected[i*2+0], :];
                chromosomeB = self.solutions[selected[i*2+1], :];
                child = crossover(chromosomeA,chromosomeB) if 0.5 < np.random.rand() else crossover(chromosomeB,chromosomeA)
                children.append(self.mutate_fcn(child))

            self.solutions = process(np.concatenate((self.elite_params, self.random_individuals_fcn(self.immigrant_popsize,self.num_params), np.array(children))))

        return self.solutions

    def tell(self, reward_table_result):
        # input must be a numpy float array
        assert(len(reward_table_result) == self.popsize), "Inconsistent reward_table size reported."

        reward_table = np.array(reward_table_result)

        if self.weight_decay > 0:
            l2_decay = compute_weight_decay(self.weight_decay, self.solutions)
            reward_table += l2_decay

        reward = reward_table
        solution = self.solutions

        reward_masked = np.ma.masked_array(reward,mask = (np.isnan(reward) | np.isinf(reward)))

        self.reward_pdf = (np.cumsum(reward_masked)/np.sum(reward_masked)).compressed()
        sorted_idx = np.argsort(reward_masked)[::-1]
        idx = sorted_idx[~reward_masked.mask][0:self.elite_popsize]

        assert(len(idx) == self.elite_popsize), "Inconsistent elite size reported."

        self.elite_rewards = reward[idx]
        self.elite_params = solution[idx]

        self.curr_best_reward = self.elite_rewards[0]

        if self.first_iteration or (self.curr_best_reward > self.best_reward):
            self.first_iteration = False
            self.best_reward = self.elite_rewards[0]
            self.best_param = np.copy(self.elite_params[0])

        if (self.sigma > self.sigma_limit):
            self.sigma *= self.sigma_decay

        self.first_iteration = False

    def current_param(self):
        return self.elite_params[0]

    def set_mu(self, mu):
        pass

    def best_param(self):
        return self.best_param

    def result(self): # return best params so far, along with historically best reward, curr reward, sigma
        return (self.best_param, self.best_reward, self.curr_best_reward, self.sigma)


def mutate(chromosome):
    mutation_rate = .5
    for j in range(chromosome.shape[0]):
        r = np.random.rand(1);
        if(r > mutation_rate):
            chromosome[j] = np.remainder(chromosome[j]+np.random.randn(1),0.99);
    return chromosome

def make_random_individuals(x,y):
    value=np.random.rand(x,y);
    return value

def boundary(individual):
    processed = individual%(1+np.finfo(float).eps)

In [17]:
class SRGAAgent():
    def __init__(self, environment):
        self._epsilon = 0.2  # 20% chances to apply a random action
        self._gamma = 0.99  # Discounting factor
        self._alpha = 0.5  # soft update param
        self.environment = environment #self._env = env
        self._resolution = .1#self.environment.resolution
        
        self.popsize=10
        self.num_paramters = 10
        self.solver = RemyGA(self.num_paramters,         # number of model parameters
                random_individuals_fcn=make_random_individuals,
                mutate_fcn = mutate,
                sigma_init=1,          # initial standard deviation
                popsize=self.popsize,       # population size
                elite_ratio=0.2,       # percentage of the elites
                forget_best=False,     # forget the historical best elites
                weight_decay=0.00,     # weight decay coefficient
                )
    def stateSpace(self):
        return range(1,self.environment.policyDimension+1)

    def train(self):
        allrewards = []
        for episode in range(20):
            rewards = []
            
            if episode % self.popsize == 0:
                # ask for a set of random candidate solutions to be evaluated
                solutions = self.solver.ask(boundary)
            
                #convert an array of 10 floats into a policy of itn, irs per year for 5 years
                policies = []
                for v in solutions:
                    actions = [i for i in itertools.zip_longest(*[iter(v)] * 2, fillvalue="")]
                    policy = {i+1: list(actions[i]) for i in range(5)}
                    policies.append(policy)

                # calculate the reward for each given solution using the environment's method
                batchRewards = self.environment.evaluatePolicy(policies)
                rewards.append(batchRewards)

                self.solver.tell(batchRewards);

                allrewards.append(rewards)
        return np.array(allrewards)

    def generate(self):
        self.train()
        #generate a policy from the array used to represent the candidate solution
        actions = [i for i in itertools.zip_longest(*[iter(self.solver.best_param)] * 2, fillvalue="")]
        best_policy = {state: list(actions[state-1]) for state in self.stateSpace()}
        best_reward = self.environment.evaluatePolicy(best_policy)
        print(best_policy, best_reward)
        return best_policy, best_reward


In [19]:
EvaluateChallengeSubmission(ChallengeSeqDecEnvironment, SRGAAgent, "SRGAAgent_20.csv")


TypeError: 'NoneType' object is not iterable

# Example submission: test.py

In [ ]:
from sys import exit, exc_info, argv
import numpy as np
import pandas as pd

!pip3 install git+https://github.com/slremy/netsapi --user --upgrade

from netsapi.challenge import *

class CustomAgent:
    def __init__(self, environment):
        self.environment = environment

    def generate(self):
        best_policy = None
        best_reward = -float('Inf')
        candidates = []
        try:
            # Agents should make use of 20 episodes in each training run, if making sequential decisions
            for i in range(20):
                self.environment.reset()
                policy = {}
                for j in range(5): #episode length
                    policy[str(j+1)]=[random.random(),random.random()]
                candidates.append(policy)
                
            rewards = self.environment.evaluatePolicy(candidates)
            best_policy = candidates[np.argmax(rewards)]
            best_reward = rewards[np.argmax(rewards)]
        
        except (KeyboardInterrupt, SystemExit):
            print(exc_info())
            
        return best_policy, best_reward

In [ ]:
EvaluateChallengeSubmission(ChallengeSeqDecEnvironment, CustomAgent, "example.csv")


In [2]:
envSeqDec = ChallengeSeqDecEnvironment(experimentCount = 10000) # Initialise a New Challenge Environment to post entire policy


In [34]:
#envSeqDec = ChallengeSeqDecEnvironment(experimentCount = 10000) 
#elite1 = {'1': [0, 1], '2': [1, 0], '3': [0, 1], '4': [1, 0], '5': [1, 1]}
elite2 = {'1': [1, 0], '2': [0, 1], '3': [1, 0], '4': [0, 1], '5': [1, 1]}
e1 = 0.05
e2 = 0.05
e3 = 0.00

elite1 = {'1': [0, 0.8], '2': [1, 0], '3': [0, 0.8 - e1], '4': [1 - e2, 0], '5': [0, 1 - e3]}
#t1 = envSeqDec.evaluatePolicy(elite2)
for _ in range(10):
    r = envSeqDec.evaluatePolicy(elite1)
    print(r)



9665  Evaluations Remaining
486.1110150009755
9660  Evaluations Remaining
476.32087756693113
9655  Evaluations Remaining
481.8689595453067
9650  Evaluations Remaining
484.60595181411634
9645  Evaluations Remaining
510.53765739300115
9640  Evaluations Remaining
514.1297390011329
9635  Evaluations Remaining
500.6721224367213
9630  Evaluations Remaining
480.77084582121046
9625  Evaluations Remaining
496.65272193377143
9620  Evaluations Remaining
505.8665283714065


In [50]:
envSeqDec = ChallengeSeqDecEnvironment(experimentCount = 10000)  
elite = {'1': [1, 0], '2': [0, 1], '3': [1, 1], '4': [1, 0], '5': [0.5, 1]}
#elite_ = {'1': [0, 1], '2': [1, 0], '3': [0, 1], '4': [1, 0], '5': [1, 1]}

elite_ = {'1': [1, 1], '2': [1, 1], '3': [1, 1], '4': [1, 1], '5': [1, 1]}
t3 = envSeqDec.evaluatePolicy(elite_)
t4 = envSeqDec.evaluatePolicy(elite)



10000  Evaluations Remaining
9995  Evaluations Remaining


In [51]:
t1 - t3

-1.7939675453223014

In [52]:
t2 - t4

2.2088338646073

In [61]:
!pip install gaft
from gaft import GAEngine
from gaft.components import BinaryIndividual, Population, DecimalIndividual
from gaft.operators import RouletteWheelSelection, UniformCrossover, FlipBitMutation, ExponentialRankingSelection
import math
from random import random, uniform
# Analysis plugin base class.
from gaft.plugin_interfaces.analysis import OnTheFlyAnalysis

In [122]:
import time

time_start=time.time()

import random
envSeqDec = ChallengeSeqDecEnvironment(experimentCount = 10000) # Initialise a New Challenge Environment to post entire policy
env = ChallengeEnvironment(experimentCount = 10000)
eps = 0.1 # equal to actions space resolution # range/eps
pop_size = 4
cross_prob = 1
exchange_prob = 1
mutation_pob = 1
generation = 4
REWARDS = []
NEW = []
POLICY = []
tmp_reward = []
tmp_policy = []

random.seed(30)

raodong = 5
elite_ = {'1': [1, 1], '2': [1, 1], '3': [1, 1], '4': [1, 1], '5': [1, 1]}
#bodong = envSeqDec.evaluatePolicy(elite_)


# 2. Define population
indv_template = DecimalIndividual(ranges=[(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1),(0, 1), (0, 1)], eps=eps)
population = Population(indv_template=indv_template, size = pop_size)
population.init()  # Initialize population with individuals.

# 3. Create genetic operators

# Use built-in operators here.
selection = RouletteWheelSelection()
crossover = UniformCrossover(pc=cross_prob, pe=exchange_prob) # PE = Gene exchange probability
mutation = FlipBitMutation(pm=mutation_pob) # 0.1 todo The probability of mutation

# 4. Create genetic algorithm engine to run optimization

engine = GAEngine(population=population, selection=selection,
                  crossover=crossover, mutation=mutation,)
                 # analysis=[FitnessStore])
    
    
# 5. Define and register fitness function   
policy = policies[0]
@engine.fitness_register
#@engine.dynamic_linear_scaling(target='max', ksi0=2, r=0.9)

def fitness(indv):
    p = [0 for _ in range(10)]
    p = indv.solution
    policy = {'1': [p[0], p[1]], '2': [p[2], p[3]], '3': [p[4], p[5]], '4': [p[6], p[7]], '5': [p[8], p[9]]}
    reward = envSeqDec.evaluatePolicy(policy) # Action in Year 1 only
    print('Sequential Result : ', reward)
    tmp_reward.append(reward)
    tmp_policy.append(policy)
    tmp_single = []
#     for i in range(5):
#         tmp_single.append(env.evaluateReward(np.asarray(policy[str(i + 1)])))
#         print('Reward of point', str(i), ' : ', tmp_single[i])
#     print('Sum of single point : ', sum(tmp_single)) 
#    NEW.append(sum(tmp))
    return reward + uniform(-raodong,raodong) 

@engine.analysis_register
class ConsoleOutput(OnTheFlyAnalysis):
    master_only = True
    interval = 1
    def register_step(self, g, population, engine):
        best_indv = population.best_indv(engine.fitness)
        msg = 'Generation: {}, best fitness: {:.3f}'.format(g + 1, engine.fmax)
        best_reward = max(tmp_reward[g : g + pop_size * generation])
        REWARDS.append(best_reward)
        #best_policy = POLICY[tmp_reward.index(best_reward)]
        #POLICY.append(best_policy)
        engine.logger.info(msg)
    
engine.run(ng = generation)
x = list(range(len(REWARDS)))
plt.plot(x, REWARDS)
plt.title('Sequential Rewards')
#plt.savefig('./res_geneticAlgorithm/Sequential_Rewards_eps:' + str(eps) +'_popsize:' + str(pop_size) +'_generation:' + str(generation) +'.jpg')
plt.show()

time_end=time.time()
print('totally cost',time_end-time_start)

10000  Evaluations Remaining
Sequential Result :  58.858378807688545
9995  Evaluations Remaining
Extra data: line 1 column 5 (char 4)
Sequential Result :  nan


ValueError: Fitness value(value: nan, type: <class 'float'>) is invalid

In [79]:
best = 0
biggest_reward = 0
candidate = [49, 55, 103]
for i in range(179,200,2):
    print(i)
    import random
    envSeqDec = ChallengeSeqDecEnvironment(experimentCount = 10000) # Initialise a New Challenge Environment to post entire policy
    env = ChallengeEnvironment(experimentCount = 10000)
    eps = 1 # equal to actions space resolution # range/eps
    pop_size = 4
    cross_prob = 1
    exchange_prob = 1
    mutation_pob = 1
    generation = 4
    REWARDS = []
    NEW = []
    POLICY = []
    tmp_reward = []
    tmp_policy = []

    random.seed(i)

    raodong = 5
    elite_ = {'1': [1, 1], '2': [1, 1], '3': [1, 1], '4': [1, 1], '5': [1, 1]}
    #bodong = envSeqDec.evaluatePolicy(elite_)


    # 2. Define population
    indv_template = DecimalIndividual(ranges=[(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1),(0, 1), (0, 1)], eps=eps)
    population = Population(indv_template=indv_template, size = pop_size)
    population.init()  # Initialize population with individuals.

    # 3. Create genetic operators

    # Use built-in operators here.
    selection = RouletteWheelSelection()
    crossover = UniformCrossover(pc=cross_prob, pe=exchange_prob) # PE = Gene exchange probability
    mutation = FlipBitMutation(pm=mutation_pob) # 0.1 todo The probability of mutation

    # 4. Create genetic algorithm engine to run optimization

    engine = GAEngine(population=population, selection=selection,
                      crossover=crossover, mutation=mutation,)
                     # analysis=[FitnessStore])


    # 5. Define and register fitness function   
    policy = policies[0]
    @engine.fitness_register
    #@engine.dynamic_linear_scaling(target='max', ksi0=2, r=0.9)

    def fitness(indv):
        p = [0 for _ in range(10)]
        p = indv.solution
        policy = {'1': [p[0], p[1]], '2': [p[2], p[3]], '3': [p[4], p[5]], '4': [p[6], p[7]], '5': [p[8], p[9]]}
        reward = envSeqDec.evaluatePolicy(policy) # Action in Year 1 only
        #print('Sequential Result : ', reward)
        tmp_reward.append(reward)
        tmp_policy.append(policy)
        tmp_single = []
    #     for i in range(5):
    #         tmp_single.append(env.evaluateReward(np.asarray(policy[str(i + 1)])))
    #         print('Reward of point', str(i), ' : ', tmp_single[i])
    #     print('Sum of single point : ', sum(tmp_single)) 
    #    NEW.append(sum(tmp))
        return reward + uniform(-raodong,raodong) 

    @engine.analysis_register
    class ConsoleOutput(OnTheFlyAnalysis):
        master_only = True
        interval = 1
        def register_step(self, g, population, engine):
            best_indv = population.best_indv(engine.fitness)
            msg = 'Generation: {}, best fitness: {:.3f}'.format(g + 1, engine.fmax)
            best_reward = max(tmp_reward[g : g + pop_size * generation])
            REWARDS.append(best_reward)
            #best_policy = POLICY[tmp_reward.index(best_reward)]
            #POLICY.append(best_policy)
            engine.logger.info(msg)

    engine.run(ng = generation)
    print(REWARDS[-1])
    if REWARDS[-1] > biggest_reward:
        best = i
    if REWARDS[-1] > 450:
        candidate.append(i)
    print(candidate)

179
10000  Evaluations Remaining
9995  Evaluations Remaining
9990  Evaluations Remaining
9985  Evaluations Remaining
9980  Evaluations Remaining
9975  Evaluations Remaining
9970  Evaluations Remaining
9965  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 1, best fitness: 312.984
9960  Evaluations Remaining
9955  Evaluations Remaining
9950  Evaluations Remaining
9945  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 2, best fitness: 299.649
9940  Evaluations Remaining
9935  Evaluations Remaining
9930  Evaluations Remaining
9925  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 3, best fitness: 402.722
9920  Evaluations Remaining
9915  Evaluations Remaining
9910  Evaluations Remaining
9905  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 4, best fitness: 407.219
405.7769967274321
[49, 55, 103]
181
10000  Evaluations Remaining
9995  Evaluations Remaining
9990  Evaluations Remaining
9985  Evaluations Remaining
9980  Evaluations Remaining
9975  Eval

gaft.GAEngine   INFO     Generation: 3, best fitness: 370.100
9920  Evaluations Remaining
9915  Evaluations Remaining
9910  Evaluations Remaining
9905  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 4, best fitness: 386.300
388.41912117993394
[49, 55, 103, 187]
199
10000  Evaluations Remaining
9995  Evaluations Remaining
9990  Evaluations Remaining
9985  Evaluations Remaining
9980  Evaluations Remaining
9975  Evaluations Remaining
9970  Evaluations Remaining
9965  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 1, best fitness: 472.349
9960  Evaluations Remaining
9955  Evaluations Remaining
9950  Evaluations Remaining
9945  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 2, best fitness: 488.685
9940  Evaluations Remaining
9935  Evaluations Remaining
9930  Evaluations Remaining
9925  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 3, best fitness: 482.735
9920  Evaluations Remaining
9915  Evaluations Remaining
9910  Evaluations Remaining
990

In [77]:
!pip install gaft --user --upgrade

Requirement already up-to-date: gaft in /Applications/anaconda3/lib/python3.7/site-packages (0.5.7)


In [86]:
best_5_runs = []
for j in range(5):
    best = 0
    biggest_reward = 0
    candidate_rewards = []
    candidate = [49, 55, 103, 187, 199, 20, 30, 54, 76, 124, 180]
    for i in candidate:
        print(i)
        import random
        envSeqDec = ChallengeSeqDecEnvironment(experimentCount = 10000) # Initialise a New Challenge Environment to post entire policy
        env = ChallengeEnvironment(experimentCount = 10000)
        eps = 1 # equal to actions space resolution # range/eps
        pop_size = 4
        cross_prob = 1
        exchange_prob = 1
        mutation_pob = 1
        generation = 4
        REWARDS = []
        NEW = []
        POLICY = []
        tmp_reward = []
        tmp_policy = []

        random.seed(i)

        raodong = 5
        elite_ = {'1': [1, 1], '2': [1, 1], '3': [1, 1], '4': [1, 1], '5': [1, 1]}
        #bodong = envSeqDec.evaluatePolicy(elite_)


        # 2. Define population
        indv_template = DecimalIndividual(ranges=[(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1),(0, 1), (0, 1)], eps=eps)
        population = Population(indv_template=indv_template, size = pop_size)
        population.init()  # Initialize population with individuals.

        # 3. Create genetic operators

        # Use built-in operators here.
        selection = RouletteWheelSelection()
        crossover = UniformCrossover(pc=cross_prob, pe=exchange_prob) # PE = Gene exchange probability
        mutation = FlipBitMutation(pm=mutation_pob) # 0.1 todo The probability of mutation

        # 4. Create genetic algorithm engine to run optimization

        engine = GAEngine(population=population, selection=selection,
                          crossover=crossover, mutation=mutation,)
                         # analysis=[FitnessStore])


        # 5. Define and register fitness function   
        policy = policies[0]
        @engine.fitness_register
        #@engine.dynamic_linear_scaling(target='max', ksi0=2, r=0.9)

        def fitness(indv):
            p = [0 for _ in range(10)]
            p = indv.solution
            policy = {'1': [p[0], p[1]], '2': [p[2], p[3]], '3': [p[4], p[5]], '4': [p[6], p[7]], '5': [p[8], p[9]]}
            reward = envSeqDec.evaluatePolicy(policy) # Action in Year 1 only
            #print('Sequential Result : ', reward)
            tmp_reward.append(reward)
            tmp_policy.append(policy)
            tmp_single = []
        #     for i in range(5):
        #         tmp_single.append(env.evaluateReward(np.asarray(policy[str(i + 1)])))
        #         print('Reward of point', str(i), ' : ', tmp_single[i])
        #     print('Sum of single point : ', sum(tmp_single)) 
        #    NEW.append(sum(tmp))
            return reward + uniform(-raodong,raodong) 

        @engine.analysis_register
        class ConsoleOutput(OnTheFlyAnalysis):
            master_only = True
            interval = 1
            def register_step(self, g, population, engine):
                best_indv = population.best_indv(engine.fitness)
                msg = 'Generation: {}, best fitness: {:.3f}'.format(g + 1, engine.fmax)
                best_reward = max(tmp_reward[g : g + pop_size * generation])
                REWARDS.append(best_reward)
                #best_policy = POLICY[tmp_reward.index(best_reward)]
                #POLICY.append(best_policy)
                engine.logger.info(msg)

        engine.run(ng = generation)
        print(REWARDS[-1])
        if REWARDS[-1] > biggest_reward:
            best = i
            biggest_reward = REWARDS[-1]
        candidate_rewards.append(REWARDS[-1])
        print(best, max(candidate_rewards))
    best_5_runs.append([best, max(candidate_rewards)])

49
10000  Evaluations Remaining
9995  Evaluations Remaining
9990  Evaluations Remaining
9985  Evaluations Remaining
9980  Evaluations Remaining
9975  Evaluations Remaining
9970  Evaluations Remaining
9965  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 1, best fitness: 473.637
9960  Evaluations Remaining
9955  Evaluations Remaining
9950  Evaluations Remaining
9945  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 2, best fitness: 494.857
9940  Evaluations Remaining
9935  Evaluations Remaining
9930  Evaluations Remaining
9925  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 3, best fitness: 495.791
9920  Evaluations Remaining
9915  Evaluations Remaining
9910  Evaluations Remaining
9905  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 4, best fitness: 489.961
494.9138397016465
49 494.9138397016465
55
10000  Evaluations Remaining
9995  Evaluations Remaining
9990  Evaluations Remaining
9985  Evaluations Remaining
9980  Evaluations Remaining
9975 

9930  Evaluations Remaining
9925  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 3, best fitness: 298.123
9920  Evaluations Remaining
9915  Evaluations Remaining
9910  Evaluations Remaining
9905  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 4, best fitness: 478.311
482.86427163388163
187 513.9350226859035
180
10000  Evaluations Remaining
9995  Evaluations Remaining
9990  Evaluations Remaining
9985  Evaluations Remaining
9980  Evaluations Remaining
9975  Evaluations Remaining
9970  Evaluations Remaining
9965  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 1, best fitness: 304.242
9960  Evaluations Remaining
9955  Evaluations Remaining
9950  Evaluations Remaining
9945  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 2, best fitness: 322.040
9940  Evaluations Remaining
9935  Evaluations Remaining
9930  Evaluations Remaining
9925  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 3, best fitness: 473.522
9920  Evaluations Remaining


9965  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 1, best fitness: 408.394
9960  Evaluations Remaining
9955  Evaluations Remaining
9950  Evaluations Remaining
9945  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 2, best fitness: 406.966
9940  Evaluations Remaining
9935  Evaluations Remaining
9930  Evaluations Remaining
9925  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 3, best fitness: 477.782
9920  Evaluations Remaining
9915  Evaluations Remaining
9910  Evaluations Remaining
9905  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 4, best fitness: 479.637
483.10019199494104
49 527.6021891007503
124
10000  Evaluations Remaining
9995  Evaluations Remaining
9990  Evaluations Remaining
9985  Evaluations Remaining
9980  Evaluations Remaining
9975  Evaluations Remaining
9970  Evaluations Remaining
9965  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 1, best fitness: 318.295
9960  Evaluations Remaining
9955  Evaluations Remaining
9

9905  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 4, best fitness: 476.432
487.66738403490933
103 514.6063844005765
54
10000  Evaluations Remaining
9995  Evaluations Remaining
9990  Evaluations Remaining
9985  Evaluations Remaining
9980  Evaluations Remaining
9975  Evaluations Remaining
9970  Evaluations Remaining
9965  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 1, best fitness: 483.670
9960  Evaluations Remaining
9955  Evaluations Remaining
9950  Evaluations Remaining
9945  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 2, best fitness: 478.118
9940  Evaluations Remaining
9935  Evaluations Remaining
9930  Evaluations Remaining
9925  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 3, best fitness: 493.556
9920  Evaluations Remaining
9915  Evaluations Remaining
9910  Evaluations Remaining
9905  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 4, best fitness: 478.365
494.56887799758397
103 514.6063844005765
76
10000  Evalua

gaft.GAEngine   INFO     Generation: 2, best fitness: 464.285
9940  Evaluations Remaining
9935  Evaluations Remaining
9930  Evaluations Remaining
9925  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 3, best fitness: 517.094
9920  Evaluations Remaining
9915  Evaluations Remaining
9910  Evaluations Remaining
9905  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 4, best fitness: 504.169
512.4893627882923
55 514.7370681304569
30
10000  Evaluations Remaining
9995  Evaluations Remaining
9990  Evaluations Remaining
9985  Evaluations Remaining
9980  Evaluations Remaining
9975  Evaluations Remaining
9970  Evaluations Remaining
9965  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 1, best fitness: 288.917
9960  Evaluations Remaining
9955  Evaluations Remaining
9950  Evaluations Remaining
9945  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 2, best fitness: 298.457
9940  Evaluations Remaining
9935  Evaluations Remaining
9930  Evaluations Remaining
992

9985  Evaluations Remaining
9980  Evaluations Remaining
9975  Evaluations Remaining
9970  Evaluations Remaining
9965  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 1, best fitness: 461.025
9960  Evaluations Remaining
9955  Evaluations Remaining
9950  Evaluations Remaining
9945  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 2, best fitness: 489.497
9940  Evaluations Remaining
9935  Evaluations Remaining
9930  Evaluations Remaining
9925  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 3, best fitness: 503.639
9920  Evaluations Remaining
9915  Evaluations Remaining
9910  Evaluations Remaining
9905  Evaluations Remaining
gaft.GAEngine   INFO     Generation: 4, best fitness: 485.116
501.9546307808688
187 520.345830216731
20
10000  Evaluations Remaining
9995  Evaluations Remaining
9990  Evaluations Remaining
9985  Evaluations Remaining
9980  Evaluations Remaining
9975  Evaluations Remaining
9970  Evaluations Remaining
9965  Evaluations Remaining
gaft.GAEn

In [87]:
best_5_runs

[[187, 513.9350226859035],
 [49, 527.6021891007503],
 [103, 514.6063844005765],
 [55, 514.7370681304569],
 [187, 520.345830216731]]

# Sarsa


In [47]:
class RL(object):
    def __init__(self, action_space, learning_rate=0.01, reward_decay=0.9, e_greedy=0.9):
        self.actions = action_space  # a list
        self.lr = learning_rate
        self.gamma = reward_decay
        self.epsilon = e_greedy

        self.q_table = pd.DataFrame(columns=self.actions, dtype=np.float64)

    def check_state_exist(self, state):
        if state not in self.q_table.index:
            # append new state to q table
            self.q_table = self.q_table.append(
                pd.Series(
                    [0]*len(self.actions),
                    index=self.q_table.columns,
                    name=state,
                )
            )

    def choose_action(self, observation):
        self.check_state_exist(observation)
        # action selection
        if np.random.rand() < self.epsilon:
            # choose best action
            state_action = self.q_table.loc[observation, :]
            # some actions may have the same value, randomly choose on in these actions
            action = np.random.choice(state_action[state_action == np.max(state_action)].index)
        else:
            # choose random action
            action = np.random.choice(self.actions)
        return action

    def learn(self, *args):
        pass


# off-policy
class QLearningTable():
    
    def __init__(self, actions, learning_rate=0.01, reward_decay=0.9, e_greedy=0.9):
        super(QLearningTable, self).__init__(actions, learning_rate, reward_decay, e_greedy)

    def learn(self, s, a, r, s_):
        self.check_state_exist(s_)
        q_predict = self.q_table.loc[s, a]
        if s_ != 'terminal':
            q_target = r + self.gamma * self.q_table.loc[s_, :].max()  # next state is not terminal
        else:
            q_target = r  # next state is terminal
        self.q_table.loc[s, a] += self.lr * (q_target - q_predict)  # update


# on-policy
class SarsaTable(RL):

    def __init__(self, actions, learning_rate=0.01, reward_decay=0.9, e_greedy=0.9):
        super(SarsaTable, self).__init__(actions, learning_rate, reward_decay, e_greedy)

    def learn(self, s, a, r, s_, a_):
        self.check_state_exist(s_)
        q_predict = self.q_table.loc[s, a]
        if s_ != 'terminal':
            q_target = r + self.gamma * self.q_table.loc[s_, a_]  # next state is not terminal
        else:
            q_target = r  # next state is terminal
        self.q_table.loc[s, a] += self.lr * (q_target - q_predict)  # update
    


In [52]:
def generate():
    
   # action_space = [[0.0, 1.0], [1.0, 0.0], [0.2, 0], [0, 0.2], [0.2, 0.2]]
    action_space = [[0.0, 0.8], [1.0, 0.0], [0.0, 0.0], [1.0, 1.0]]
    rewards_20 = []
    policy_20 = []
    rewards_seq = []
    envSeqDec = ChallengeSeqDecEnvironment(experimentCount=20000)

    for episode in range(20):
        
        # initial observation
        envSeqDec.reset()
        observation =1
        rewards=0
        policy={}
        
        for j in range(5):
            # fresh env
            # env.render()
            # RL choose action based on observation
            # print(observation)
            # print(str(observation))
            action = RL.choose_action(str(observation))
            a=action_space[action-1]
            #print(';;;;;;;;;;;',j,']]]]',a)
            policy[str(j+1)]=a
            # RL take action and get next observation and reward
            observation_, reward, done, info = envSeqDec.evaluateAction(a)
            if reward:
                rewards+=reward
            if not reward:
                pass
            #print(observation_, reward, done, info)
            action_ = RL.choose_action(str(observation_))
            
            # RL learn from this transition
            RL.learn(str(observation), action, reward, str(observation_), action_)
            
            # swap observation
            observation = observation_
            
            # break while loop when end of this episode
            if done:
                #print('Episode:', episode + 1, ' Reward: %i' % int(rewards))
                #print('Policy:', policy)
                break
                
        #print('sequential result')
        #seq_reward = envSeqDec.evaluatePolicy(policy)
        #rewards_seq.append(seq_reward)
        #print(seq_reward)
        rewards_20.append(rewards)
        policy_20.append(policy)
    if(max(rewards_20)) > 450:
        print(f'*******************************got it {ii}_{policy}_{max(rewards_20)}')
    print(max(rewards_20))
   # print('Best Reward:',np.max(rewards_20))
   # print('Best Policy:',policy_20[np.argmax(rewards_20)])
#    x = list(range(len(rewards_20)))
#     plt.plot(x, rewards_20)
#     #plt.title(f'Sarsa Result action_space: {action_space} learn_rate: {learning_rate} reward_decay: {reward_decay} e_greedy: {e_greedy}')
#     plt.title('Sarsa Result')
#     plt.xlabel('Episodes')
#     plt.ylabel('Rewards')
#     plt.show()


In [53]:
envSeqDec = ChallengeSeqDecEnvironment(experimentCount=20000)
action_space = [1,2,3,4]
RL = SarsaTable(actions=action_space,
                learning_rate=0.03,
                reward_decay=0.4,
                e_greedy=0.9)
generate()

20000  Evaluations Remaining
19999  Evaluations Remaining


KeyboardInterrupt: 

In [54]:
def run_this(ii):
    random.seed(ii)
    print('sssssssssssssssssssssssssssssssssssssseed', ii)
    action_space = [1,2]
    RL = SarsaTable(actions=action_space,
                    learning_rate=0.03,
                    reward_decay=0.4,
                    e_greedy=0.9)
    generate()

In [55]:
import time
def main():
    
    index = list(range(1, 200))
    pool = Pool()
    func = run_this
    pool.map(func, index)
    pool.close()
    pool.join()

    
print(f'搜索开始时间：{time.strftime("%Y-%m-%d %H:%M:%S")}')
anss = {}
main()
print(f'搜索结束时间：{time.strftime("%Y-%m-%d %H:%M:%S")}')


搜索开始时间：2019-06-27 10:27:44
sssssssssssssssssssssssssssssssssssssseed 1
sssssssssssssssssssssssssssssssssssssseed 16
sssssssssssssssssssssssssssssssssssssseed 21
sssssssssssssssssssssssssssssssssssssseed 11
sssssssssssssssssssssssssssssssssssssseed 26
sssssssssssssssssssssssssssssssssssssseed 6
sssssssssssssssssssssssssssssssssssssseed 31
sssssssssssssssssssssssssssssssssssssseed 41
sssssssssssssssssssssssssssssssssssssseed 36
sssssssssssssssssssssssssssssssssssssseed 46
sssssssssssssssssssssssssssssssssssssseed 56
sssssssssssssssssssssssssssssssssssssseed 51
20000  Evaluations Remaining
20000  Evaluations Remaining
20000  Evaluations Remaining
20000  Evaluations Remaining
20000  Evaluations Remaining
20000  Evaluations Remaining
20000  Evaluations Remaining
20000  Evaluations Remaining
20000  Evaluations Remaining
20000  Evaluations Remaining
20000  Evaluations Remaining
20000  Evaluations Remaining
19999  Evaluations Remaining
19999  Evaluations Remaining
19999  Evaluations Remaining


19978  Evaluations Remaining
19979  Evaluations Remaining
19978  Evaluations Remaining
19978  Evaluations Remaining
19978  Evaluations Remaining
19978  Evaluations Remaining
19978  Evaluations Remaining
19979  Evaluations Remaining
19978  Evaluations Remaining
19978  Evaluations Remaining
19977  Evaluations Remaining
19977  Evaluations Remaining
19977  Evaluations Remaining
19977  Evaluations Remaining
19978  Evaluations Remaining
19977  Evaluations Remaining
19977  Evaluations Remaining
19977  Evaluations Remaining
19978  Evaluations Remaining
19977  Evaluations Remaining
19977  Evaluations Remaining
19977  Evaluations Remaining
19976  Evaluations Remaining
19976  Evaluations Remaining
19976  Evaluations Remaining
19976  Evaluations Remaining
19977  Evaluations Remaining
19976  Evaluations Remaining
19976  Evaluations Remaining
19976  Evaluations Remaining
19976  Evaluations Remaining
19976  Evaluations Remaining
19977  Evaluations Remaining
19976  Evaluations Remaining
19975  Evaluat

19956  Evaluations Remaining
19956  Evaluations Remaining
19954  Evaluations Remaining
19954  Evaluations Remaining
19954  Evaluations Remaining
19954  Evaluations Remaining
19955  Evaluations Remaining
19955  Evaluations Remaining
19953  Evaluations Remaining
19954  Evaluations Remaining
19954  Evaluations Remaining
19953  Evaluations Remaining
19955  Evaluations Remaining
19953  Evaluations Remaining
19952  Evaluations Remaining
19954  Evaluations Remaining
19954  Evaluations Remaining
19954  Evaluations Remaining
19953  Evaluations Remaining
19953  Evaluations Remaining
19954  Evaluations Remaining
19952  Evaluations Remaining
19952  Evaluations Remaining
19954  Evaluations Remaining
19951  Evaluations Remaining
19953  Evaluations Remaining
19953  Evaluations Remaining
19953  Evaluations Remaining
19952  Evaluations Remaining
19953  Evaluations Remaining
19953  Evaluations Remaining
19951  Evaluations Remaining
19951  Evaluations Remaining
19954  Evaluations Remaining
19952  Evaluat

19931  Evaluations Remaining
19934  Evaluations Remaining
19931  Evaluations Remaining
19929  Evaluations Remaining
19930  Evaluations Remaining
19931  Evaluations Remaining
19930  Evaluations Remaining
19933  Evaluations Remaining
19929  Evaluations Remaining
19933  Evaluations Remaining
19931  Evaluations Remaining
19928  Evaluations Remaining
19930  Evaluations Remaining
19933  Evaluations Remaining
19930  Evaluations Remaining
19928  Evaluations Remaining
19929  Evaluations Remaining
19930  Evaluations Remaining
19929  Evaluations Remaining
19928  Evaluations Remaining
19932  Evaluations Remaining
19930  Evaluations Remaining
19932  Evaluations Remaining
19927  Evaluations Remaining
19929  Evaluations Remaining
19932  Evaluations Remaining
19929  Evaluations Remaining
19927  Evaluations Remaining
19928  Evaluations Remaining
19929  Evaluations Remaining
19928  Evaluations Remaining
19927  Evaluations Remaining
19931  Evaluations Remaining
19931  Evaluations Remaining
19929  Evaluat

19907  Evaluations Remaining
19908  Evaluations Remaining
19907  Evaluations Remaining
19910  Evaluations Remaining
19907  Evaluations Remaining
19905  Evaluations Remaining
19910  Evaluations Remaining
19908  Evaluations Remaining
19905  Evaluations Remaining
19905  Evaluations Remaining
19906  Evaluations Remaining
19907  Evaluations Remaining
19906  Evaluations Remaining
19909  Evaluations Remaining
19909  Evaluations Remaining
19907  Evaluations Remaining
19904  Evaluations Remaining
19906  Evaluations Remaining
19909  Evaluations Remaining
19905  Evaluations Remaining
19906  Evaluations Remaining
19904  Evaluations Remaining
19908  Evaluations Remaining
19906  Evaluations Remaining
19903  Evaluations Remaining
19908  Evaluations Remaining
19905  Evaluations Remaining
19905  Evaluations Remaining
19904  Evaluations Remaining
19905  Evaluations Remaining
19908  Evaluations Remaining
19907  Evaluations Remaining
19905  Evaluations Remaining
19907  Evaluations Remaining
19904  Evaluat

...

19962  Evaluations Remaining
19961  Evaluations Remaining
19959  Evaluations Remaining
19964  Evaluations Remaining
19959  Evaluations Remaining
19963  Evaluations Remaining
19965  Evaluations Remaining
19960  Evaluations Remaining
19961  Evaluations Remaining
19963  Evaluations Remaining
19961  Evaluations Remaining
19965  Evaluations Remaining
19961  Evaluations Remaining
19963  Evaluations Remaining
19960  Evaluations Remaining
19958  Evaluations Remaining
19958  Evaluations Remaining
19962  Evaluations Remaining
19964  Evaluations Remaining
19959  Evaluations Remaining
19962  Evaluations Remaining
19960  Evaluations Remaining
sssssssssssssssssssssssssssssssssssssseed 181
sssssssssssssssssssssssssssssssssssssseed 186
sssssssssssssssssssssssssssssssssssssseed 96
20000  Evaluations Remaining
sssssssssssssssssssssssssssssssssssssseed 101
20000  Evaluations Remaining
20000  Evaluations Remaining
20000  Evaluations Remaining
sssssssssssssssssssssssssssssssssssssseed 191
sssssssssssssssss

Process ForkPoolWorker-555:
Process ForkPoolWorker-590:
Process ForkPoolWorker-588:
Process ForkPoolWorker-583:
Process ForkPoolWorker-586:
Process ForkPoolWorker-578:
Process ForkPoolWorker-565:
Process ForkPoolWorker-595:
Process ForkPoolWorker-563:
Process ForkPoolWorker-556:
Process ForkPoolWorker-585:
Process ForkPoolWorker-582:
Process ForkPoolWorker-580:
Process ForkPoolWorker-566:
Process ForkPoolWorker-581:
Process ForkPoolWorker-564:
Process ForkPoolWorker-589:
Process ForkPoolWorker-591:
Process ForkPoolWorker-584:
Process ForkPoolWorker-594:
Process ForkPoolWorker-592:
Process ForkPoolWorker-593:
Process ForkPoolWorker-587:
Process ForkPoolWorker-579:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (mo

KeyboardInterrupt: 

  File "/Applications/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Applications/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Applications/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Applications/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/Applications/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Applications/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Applications/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/Applications/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    se

  File "/Applications/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/Applications/anaconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/Applications/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/Applications/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/Applications/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/Applications/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/Applications/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/Applications/anaconda3/lib/pytho

In [32]:
[197, 193, 189, 185, 181, 177, 173, 169, 165, 161, 157]

[197, 193, 189, 185, 181, 177, 173, 169, 165, 161, 157]

In [41]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
